In [1]:
# %%
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gc
import memory_profiler
import numpy
# import cupy as np
import matplotlib.pyplot as plt
from matplotlib import style
#style.use('classic')
import PIL
import scipy
import more_itertools
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import cv2 
import glob
import os
import time
from numpy import genfromtxt, asarray, savez_compressed, load
from sklearn.model_selection import train_test_split
from PIL import Image 
from io import StringIO

ModuleNotFoundError: No module named 'matplotlib'

## Import Tensorflow Libraries

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model

try:
  IN_COLAB = True
  from google.colab import drive
  from tensorflow.keras.optimizers.legacy import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax
except:
  IN_COLAB = False
  from tensorflow.keras.optimizers import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax

from tensorflow.keras.optimizers.legacy import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.keras.utils import plot_model, to_categorical, normalize
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()



Found GPU at: /device:GPU:0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2673883699199238801
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4841275392
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17970183833834411098
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]
1 Physical GPUs, 1 Logical GPU


### Check for available GPUs and configure Tensorflow session to utilize GPU

In [ ]:

try:
  # tf.debugging.experimental.enable_dump_debug_info('.', tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
  # tf.debugging.set_log_device_placement(True)
  from tensorflow.python.client import device_lib

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  config.gpu_options.per_process_gpu_memory_fraction = 0.1
  sess = tf.compat.v1.InteractiveSession(config=config)
  set_session(sess)
  print(device_lib.list_local_devices())
  gpus = tf.config.experimental.list_physical_devices('GPU')
  for gpu in gpus:
    try:
    
      tf.config.experimental.set_memory_growth(gpu, True)
      # Restrict TensorFlow to only use the first GPU
      tf.config.set_visible_devices(gpus[0], 'GPU')
      logical_gpus = tf.config.list_logical_devices('GPU')
      print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
      # Visible devices must be set before GPUs have been initialized
      print(e)

except Exception as error:
    print("Error trying to configure computing device.")
    print(error)

In [ ]:
if IN_COLAB:
  DATASET_FOLDER = './data/dataset1/'
  DATA_FOLDER = './data/'
else:
  DATASET_FOLDER = './../data/dataset1/'
  DATA_FOLDER = './../data/'

INPUT_DATA = 'Image_Input'
OUTPUT_DATA = 'Image_Output'


In [ ]:
files=glob.glob(str(image_path + (f'*{param_type}{short}*.png')))
        flen = len(files)
        
        if flen != 3:
            for file in files:
                #os.remove(file)
                raise Exception(f'Wrong file count: Expected 3 files but got {flen} {file}. Please rectify')
           

            
        if i%2 is 1:
            for myFile in files:
                #print(myFile)
                image=Image.open(myFile)
                image = cv2.imread (myFile)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = image.reshape(image.shape[0],image.shape[1],image.shape[2])
                #plt.imshow(image)
                #plt.show()
                if image.shape != (173, 267, 3):
                    print(myFile)
                    plt.imshow(image)
                    plt.show()
                    print(image.shape)
                    #os.remove(myFile)
                    prepData()
                    break
                im_data.append(np.asnumpy(image))
                proceed = True
            #print(count)
    